In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

import numpy as np
import pandas as pd

import seaborn as sns 
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize']=(10,10)

import warnings
warnings.filterwarnings(action='ignore')

## 1. 데이터 불러오기

In [ ]:
raw_data = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv')
raw_data

,날짜,현재수위,유입량,저수량,공용량,방류량,강화대교_조위,청담대교_유량,잠수교_유량,한강대교_유량,행주대교_유량,대곡교_강수량,진관교_강수량,송정동_강수량,청담대교_수위,잠수교_수위,한강대교_수위,행주대교_수위
0,2012-05-01 00:00,24.800,555.00,219.07,24.93,555.00,445.0,469.05,0.0,729.80,540.18,0.0,0.0,0.0,310.7,300.2,290.0,275.3
1,2012-05-01 00:10,24.794,464.60,218.86,25.15,562.90,449.0,498.00,0.0,731.48,540.18,0.0,0.0,0.0,314.7,300.2,290.0,275.3
2,2012-05-01 00:20,24.789,478.10,218.69,25.31,576.40,451.0,490.68,0.0,726.42,540.18,0.0,0.0,0.0,313.7,301.2,290.0,275.3
3,2012-05-01 00:30,24.789,464.80,218.69,25.31,563.10,452.0,476.21,0.0,726.42,552.17,0.0,0.0,0.0,311.7,301.2,290.0,276.3
4,2012-05-01 00:40,24.789,478.10,218.69,25.31,576.40,450.0,476.21,0.0,707.17,564.29,0.0,0.0,0.0,311.7,301.2,291.0,277.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276331,2022-07-18 23:10,25.040,259.23,212.86,31.14,259.23,510.0,319.84,NaN,-456.41,974.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276332,2022-07-18 23:20,25.040,260.46,212.86,31.14,260.46,492.0,314.01,NaN,-717.30,1006.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276333,2022-07-18 23:30,25.040,259.37,212.86,31.14,259.37,475.0,387.55,NaN,-843.37,1039.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276334,2022-07-18 23:40,25.040,259.13,212.86,31.14,259.13,458.0,454.91,NaN,-1023.37,1073.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
raw_data.isnull().sum()

날짜              0
현재수위          743
유입량           743
저수량           743
공용량           743
방류량           743
강화대교_조위      4927
청담대교_유량     16380
잠수교_유량     196849
한강대교_유량      1279
행주대교_유량        59
대곡교_강수량         0
진관교_강수량         0
송정동_강수량         0
청담대교_수위        59
잠수교_수위         59
한강대교_수위        59
행주대교_수위        59
dtype: int64

#팔당댐 결측치 처리 - 보간

In [ ]:
팔당처리 = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/0822_팔당댐_Arima.csv', encoding = 'cp949', usecols=[1,2,3])
팔당처리

,공용량,현재수위,방류량
0,24.93,24.800,555.00
1,25.15,24.794,562.90
2,25.31,24.789,576.40
3,25.31,24.789,563.10
4,25.31,24.789,576.40
...,...,...,...
276331,31.14,25.040,259.23
276332,31.14,25.040,260.46
276333,31.14,25.040,259.37
276334,31.14,25.040,259.13


In [ ]:
팔당처리df = pd.concat([raw_data.drop(팔당처리.columns, axis = 1), 팔당처리], axis=1)
팔당처리df

,날짜,유입량,저수량,강화대교_조위,청담대교_유량,잠수교_유량,한강대교_유량,행주대교_유량,대곡교_강수량,진관교_강수량,송정동_강수량,청담대교_수위,잠수교_수위,한강대교_수위,행주대교_수위,공용량,현재수위,방류량
0,2012-05-01 00:00,555.00,219.07,445.0,469.05,0.0,729.80,540.18,0.0,0.0,0.0,310.7,300.2,290.0,275.3,24.93,24.800,555.00
1,2012-05-01 00:10,464.60,218.86,449.0,498.00,0.0,731.48,540.18,0.0,0.0,0.0,314.7,300.2,290.0,275.3,25.15,24.794,562.90
2,2012-05-01 00:20,478.10,218.69,451.0,490.68,0.0,726.42,540.18,0.0,0.0,0.0,313.7,301.2,290.0,275.3,25.31,24.789,576.40
3,2012-05-01 00:30,464.80,218.69,452.0,476.21,0.0,726.42,552.17,0.0,0.0,0.0,311.7,301.2,290.0,276.3,25.31,24.789,563.10
4,2012-05-01 00:40,478.10,218.69,450.0,476.21,0.0,707.17,564.29,0.0,0.0,0.0,311.7,301.2,291.0,277.3,25.31,24.789,576.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276331,2022-07-18 23:10,259.23,212.86,510.0,319.84,NaN,-456.41,974.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.14,25.040,259.23
276332,2022-07-18 23:20,260.46,212.86,492.0,314.01,NaN,-717.30,1006.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.14,25.040,260.46
276333,2022-07-18 23:30,259.37,212.86,475.0,387.55,NaN,-843.37,1039.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.14,25.040,259.37
276334,2022-07-18 23:40,259.13,212.86,458.0,454.91,NaN,-1023.37,1073.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.14,25.040,259.13


#시간차 적용을 위한 4월 데이터(조위df) 불러오기

In [ ]:
from glob import glob

w_list = sorted(glob("/content/drive/MyDrive/팔당댐 데이콘_개인/dam/*.csv"))
w_list

['/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2012.csv',
 '/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2013.csv',
 '/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2014.csv',
 '/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2015.csv',
 '/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2016.csv',
 '/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2017.csv',
 '/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2018.csv',
 '/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2019.csv',
 '/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2020.csv',
 '/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2021.csv',
 '/content/drive/MyDrive/팔당댐 데이콘_개인/dam/2022.csv']

In [ ]:
idx = [0]
for i in range(287):
  idx.append(i)

In [ ]:
idx2 = []
for i in range(12):
  idx2.append(i*288)

In [ ]:
tmp3 = []

for i in range(11):
  a = pd.read_csv(w_list[i], encoding='cp949', usecols = [0,3])
  tmp3.append(a.iloc[idx].set_index(pd.Index(list(range(i+idx2[i],i+idx2[i]+288)))))

팔당df = pd.concat([tmp3[0],tmp3[1],tmp3[2],tmp3[3],tmp3[4],tmp3[5],tmp3[6],tmp3[7],tmp3[8],tmp3[9],tmp3[10]])
팔당df.columns = ['날짜', '방류량']
팔당df

,날짜,방류량
0,2012-04-29 0:00,444.0
1,2012-04-29 0:00,444.0
2,2012-04-29 0:10,531.0
3,2012-04-29 0:20,537.6
4,2012-04-29 0:30,544.2
...,...,...
3173,2022-04-30 23:00,124.66
3174,2022-04-30 23:10,124.55
3175,2022-04-30 23:20,124.48
3176,2022-04-30 23:30,124.15


In [ ]:
팔당df[팔당df['방류량']=='-']

,날짜,방류량
1087,2015-04-30 12:30,-
2115,2019-04-29 15:10,-
2988,2022-04-29 16:10,-


In [ ]:
팔당df.loc[1087]['방류량'] = 168.2
팔당df.loc[2115]['방류량'] = 368
팔당df.loc[2988]['방류량'] = 125.1

#다리별 시간차 계산

In [ ]:
idx3 = []
for i in range(12):
  idx3.append(i*26496)

In [ ]:
ttmp = []
for i in range(0, 288): 
  
  push_num = i
  
  tmp = []
  for j in range(11):
    if j != 10:
      tmp.append(pd.concat([팔당df[idx2[j+1]-push_num:idx2[j+1]],팔당처리df[idx3[j]:idx3[j+1]-push_num][['날짜','방류량']]], ignore_index=True))
    else:
      tmp.append(pd.concat([팔당df[idx2[j+1]-push_num:idx2[j+1]],팔당처리df[idx3[j]:-push_num][['날짜','방류량']]], ignore_index=True))

  push_result = pd.concat(
        [tmp[0],tmp[1],tmp[2],tmp[3],tmp[4],tmp[5],tmp[6],tmp[7],tmp[8],tmp[9],tmp[10]], ignore_index=True
    )

  mm = pd.concat([push_result[['방류량']],팔당처리df[['청담대교_수위','잠수교_수위', '한강대교_수위', '행주대교_수위']]], axis=1)
  mm['방류량'] = mm['방류량'].astype(float, errors = 'raise')
  nn = mm[:269424]
  push_cor2 = nn.corr()
  
  
  #print(i, push_cor2.iloc[:,0][2]) #마지막 원소 변경 1,2,3,4
  #ttmp.append(push_cor2.iloc[:,0][2])

  #1번: 청담대교 15 0.8970553300325268 -> 15 0.8970394478572508
  #2번: 잠수교   19 0.8748510916903229
  #3번: 한강대교 20 0.8635804208734675
  #4번: 행주대교 21 0.8088005740767352
  #시간적 순서가 직관과 맞음

  #하나만 꼽으라면 19
  print(i, push_cor2.sum()[0])
  ttmp.append(push_cor2.sum()[0])

0 4.410514618207634
1 4.410977499356765
2 4.413932674737623
3 4.416838990214302
4 4.419693759567066
5 4.422489157942971
6 4.42520050657644
7 4.427807333167553
8 4.430262776989767
9 4.4325569618259095
10 4.434684489360465
11 4.436602126759455
12 4.438296083273169
13 4.439771608897166
14 4.4410140958053566
15 4.442024023217969
16 4.4427827623522544
17 4.443323415998397
18 4.443595953155796
19 4.443641125815918
20 4.443445691407384
21 4.443002975655779
22 4.442323804839731
23 4.441424786241807
24 4.440315399191794
25 4.438978657118048
26 4.437434711186554
27 4.435696829820653
28 4.433777575205552
29 4.431694220025834
30 4.429434665028613
31 4.427017560080587
32 4.424450820136281
33 4.421757477238371
34 4.418929746779365
35 4.415987918340716
36 4.412930211449392
37 4.409768860044382
38 4.406516512427415
39 4.403179763650012
40 4.399769760037632
41 4.396277205125924
42 4.392694766799659
43 4.3890355755379975
44 4.385320144710435
45 4.381555464907175
46 4.377726044367163
47 4.373822121692718

In [ ]:
max(ttmp)  

4.443641125815918

#파일 저장

In [ ]:
push_num = 19
  
tmp = []
for j in range(11):
  if j != 10:
    tmp.append(pd.concat([팔당df[idx2[j+1]-push_num:idx2[j+1]],팔당처리df[idx3[j]:idx3[j+1]-push_num][['날짜','방류량']]], ignore_index=True))
  else:
    tmp.append(pd.concat([팔당df[idx2[j+1]-push_num:idx2[j+1]],팔당처리df[idx3[j]:-push_num][['날짜','방류량']]], ignore_index=True))

push_result = pd.concat(
      [tmp[0],tmp[1],tmp[2],tmp[3],tmp[4],tmp[5],tmp[6],tmp[7],tmp[8],tmp[9],tmp[10]], ignore_index=True
)

mm = pd.concat([push_result[['방류량']],팔당처리df[['청담대교_수위','잠수교_수위', '한강대교_수위', '행주대교_수위']]], axis=1)
mm['방류량'] = mm['방류량'].astype(float, errors = 'raise')
nn = mm[:269424]
push_cor2 = nn.corr()
push_cor2

,방류량,청담대교_수위,잠수교_수위,한강대교_수위,행주대교_수위
방류량,1.000000,0.896613,0.874833,0.863561,0.808633
청담대교_수위,0.896613,1.000000,0.991150,0.990986,0.949497
잠수교_수위,0.874833,0.991150,1.000000,0.995218,0.957848
한강대교_수위,0.863561,0.990986,0.995218,1.000000,0.972335
행주대교_수위,0.808633,0.949497,0.957848,0.972335,1.000000


In [ ]:
result = push_result.방류량
result

0          575.7
1          575.3
2          555.0
3          575.7
4          575.4
           ...  
276331    258.67
276332    258.58
276333    258.56
276334    258.65
276335    258.09
Name: 방류량, Length: 276336, dtype: object

In [ ]:
result.to_csv("방류량 완성.csv", index = False, encoding = 'cp949', columns = ['방류량'])

In [ ]:
result.isnull().sum()

0